<a href="https://colab.research.google.com/github/jw3215/006732/blob/master/Chatper3/Chapter3-2_%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습: Zero-shot Classification

이번 실습에서는 open LLM을 가지고 zero-shot classification을 해봅니다. 먼저 필요한 library들을 설치합시다.

In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00


그 다음 Gemma-2B를 사용하기 위해 다음과 같은 작업을 진행합니다:
1. huggingface.co 계정 만들고 로그인하기
2. https://www.kaggle.com/models/google/gemma/license/consent 에서 Gemma license 동의하기
3. 홈 화면으로 돌아와, `Profile > Settings > Access Tokens` 메뉴로 들어와 "Write" type의 token 생성하기
4. 생성한 토큰을 아래 "HF TOKEN"에 불여넣고 셀을 실행하기.

In [2]:
from huggingface_hub import login
from google.colab import userdata


login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


정상적으로 token을 생성하고 Gemma license에 동의했다면 아래 코드로 tokenizer와 Gemma-2B 모델을 불러올 수 있습니다.

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

이번에는 Gemma-2B를 가지고 간단한 text 생성을 해봅시다.
"What is your name?" 이라는 text를 넣었을 때 어떤 text가 생성되는지 살펴봅시다.

In [4]:
input_text = "What is your name?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>What is your name?

What is your age?

What is your gender?

What


2B의 작은 LLM이라 질좋은 답변이 나오지 않는 것을 알 수 있습니다.
이번에는 입력으로 넣어준 token들의 logit을 계산해봅시다.

In [5]:
tokens = input_ids['input_ids']
print(tokens)

logits = model(**input_ids).logits
for i in range(tokens.shape[-1]):
    token = tokens[0, i].item()
    print(logits[0, i, token])

tensor([[     2,   1841,    603,    861,   1503, 235336]], device='cuda:0')
tensor(-18.2747, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-33.2665, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-23.9536, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-27.7627, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-19.6064, device='cuda:0', grad_fn=<SelectBackward0>)
tensor(-21.0372, device='cuda:0', grad_fn=<SelectBackward0>)


위와 같이 모델 출력의 `.logits`을 통해 token들의 logit을 알 수 있습니다.
Logit은 높을 수록 token이 나올 확률이 높다는 뜻입니다.

이번에는 logit 계산을 통해 zero-shot classification을 구현해보도록 하겠습니다.

In [6]:
import torch

def zero_shot_classification(text, task_description, labels):  # text는 주어진 입력, task_description은 task에 대한 설명, labels은 class들을 text로 변환한 결과입니다.
    text_ids = tokenizer(task_description + text, return_tensors="pt").to("cuda")  # 먼저 task_description과 text를 이어붙인 후, tokenize합니다.
    probs = []
    for label in labels:  # 그 다음 각 text화된 label들을 tokenize하고 입력에 이어붙인 후, Gemma-2B에 넣어줍니다.
        label_ids = tokenizer(label, return_tensors="pt").to("cuda")
        n_label_tokens = label_ids['input_ids'].shape[-1] - 1  # text로 변환한 label의 token 수를 계산합니다.
        input_ids = {
            'input_ids': torch.concatenate([text_ids['input_ids'], label_ids['input_ids'][:, 1:]], axis=-1),  # concatenate 명령어를 통해 이어붙이는 모습입니다.
            'attention_mask': torch.concatenate([text_ids['attention_mask'], label_ids['attention_mask'][:, 1:]], axis=-1)
        }

        logits = model(**input_ids).logits  # Logit을 계산한 모습입니다.
        prob = 0
        n_total = input_ids['input_ids'].shape[-1]
        for i in range(n_label_tokens, 0, -1):  # 일반적으로 text로 변환한 label은 여러 token으로 이루어져있습니다. 이러한 label에 대한 logit은 구성하는 모든 token들의 logit들의 합으로 정의합니다.
            token = label_ids['input_ids'][0, i].item()
            prob += logits[0, n_total - i, token].item()
        probs.append(prob)

        del input_ids
        del logits
        torch.cuda.empty_cache()  # 위의 del과 empty_cache() 명령어를 통해 GPU를 제때 할당해제 해줍니다. 만약 GPU가 여유롭다면 지워주시는게 속도적으로 이득입니다.

    return probs

아래는 실제로 zero-shot classification을 해본 결과입니다.

In [7]:
probs = zero_shot_classification("I am happy!", "Is the sentence positive or negative?: ", ["positive", "negative"])
print(probs)

[-4.5151824951171875, -9.59005069732666]


보시다시피 우리는 Gemma를 별도로 학습하지 않았음에도 불구하고 주어진 문장이 긍정적이라는 것을 정확하게 예측하고 있습니다.

다음은 영화 리뷰 감정 분석 task에 적용해봅시다.
먼저 data를 불러옵니다.

In [8]:
from datasets import load_dataset


imdb = load_dataset("imdb")
def preprocess_function(examples):
    return tokenizer(examples["text"], max_length=200, truncation=True)

tokenized_imdb = imdb.map(preprocess_function, batched=True)

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

그리고 `test` data에서 50개의 영화 리뷰에 대해 예측하는 코드는 다음과 같습니다.

In [9]:
import numpy as np
from tqdm import tqdm


n_corrects = 0
for i in tqdm(range(50)):
    text = tokenized_imdb['test'][i]['text']
    label = tokenized_imdb['test'][i]['label']
    probs = zero_shot_classification(
        text,
        "A movie review is given. Decide that the movie review is positive or negative: ",
        labels=["Answer: negative.", "Answer: positive."]
    )

    pred = np.argmax(np.array(probs))
    if pred == label:
        n_corrects += 1

print(n_corrects)

100%|██████████| 50/50 [00:49<00:00,  1.01it/s]

44


보시다시피 정확도 88%로, 매우 높은 성능을 보이는 것을 알 수 있습니다.

# 기본과제

## 목표

---

이번 과제에서는 이전 주차 과제에서 활용했던 `fancyzhx/ag_news` 문제를 zero-shot classification으로 푸시면 됩니다. 아래 사항들에 유의하시면 될 것 같습니다.

- [ ]  Label들을 올바르게 text화 하여 넘겨주셔야 합니다.
- [ ]  `test` split data 50개에 대한 정확도 계산 코드 및 출력이 남아있어야 합니다.

이외에는 Gemma-2B 모델의 logit 계산 능력을 활용한다는 부분 빼고는 제약이 없습니다.

## 코드

In [3]:
from huggingface_hub import login
from google.colab import userdata


login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### 모델 & 토크나이저

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### 데이터 셋

In [5]:
from datasets import load_dataset


ag_news = load_dataset("fancyzhx/ag_news")
def preprocess_function(examples):
    return tokenizer(examples["text"], max_length=200, truncation=True)

tokenized_ag_news = ag_news.map(preprocess_function, batched=True)

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [6]:
tokenized_ag_news

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7600
    })
})

In [7]:
classes = ag_news['train'].features['label'].names
print(classes)
ag_news_labels = [f"Answer: {cls}." for cls in classes]
print(ag_news_labels)

['World', 'Sports', 'Business', 'Sci/Tech']
['Answer: World.', 'Answer: Sports.', 'Answer: Business.', 'Answer: Sci/Tech.']


### 분류 함수

In [9]:
import torch

def zero_shot_classification(text, task_description, labels):
    text_ids = tokenizer(task_description + text, return_tensors="pt").to("cuda")
    probs = []
    for label in labels:
        label_ids = tokenizer(label, return_tensors="pt").to("cuda")
        n_label_tokens = label_ids['input_ids'].shape[-1] - 1
        input_ids = {
            'input_ids': torch.concatenate([text_ids['input_ids'], label_ids['input_ids'][:, 1:]], axis=-1),
            'attention_mask': torch.concatenate([text_ids['attention_mask'], label_ids['attention_mask'][:, 1:]], axis=-1)
        }

        logits = model(**input_ids).logits
        prob = 0
        n_total = input_ids['input_ids'].shape[-1]
        for i in range(n_label_tokens, 0, -1):
            token = label_ids['input_ids'][0, i].item()
            prob += logits[0, n_total - i, token].item()
        probs.append(prob)

        del input_ids
        del logits
        torch.cuda.empty_cache()

    return probs

### 분류 수행

In [11]:
import numpy as np
from tqdm import tqdm

n_corrects = 0
n_total = 50
for i in tqdm(range(n_total)):
    text = tokenized_ag_news['test'][i]['text']
    label = tokenized_ag_news['test'][i]['label']
    probs = zero_shot_classification(
        text,
        "A news article is given. Decide that the news article is about: ",
        labels=ag_news_labels
    )

    pred = np.argmax(np.array(probs))
    if pred == label:
        n_corrects += 1

print('\n\n')
print(f"Corrects: {n_corrects}/{n_total}")

100%|██████████| 50/50 [00:37<00:00,  1.32it/s]




Corrects: 24/50


## 결과

- 정확도: 48%